In [1]:
import xarray as xr
import geocat.comp as gc
import numpy as np

In [2]:
conda list geocat

# packages in environment at /glade/work/oero/miniconda3/envs/geocat-notebooks:
#
# Name                    Version                   Build  Channel
geocat-comp               2022.04.0          pyha770c72_0    conda-forge
geocat-datafiles          2022.03.0          pyha770c72_0    conda-forge
geocat-f2py               2021.04.0        py38h3d0eb6f_0    ncar
geocat-viz                2022.05.0          pyhd8ed1ab_0    conda-forge

Note: you may need to restart the kernel to use updated packages.


In [3]:
isccp_climo_filename = "ISCCP-Basic.HGG.GLOBAL.10KM.climo.198307-201706.nc"

nds = xr.open_dataset("/glade/work/brianpm/observations/isccp/climo/"+isccp_climo_filename)

x = nds['CLDTOT_ISCCP']

# 2° grid
olon = xr.DataArray(np.linspace(0, 358, 180), dims=["lon"])
olon.name = "lon"
olon.attrs["units"] = "degrees_east"
olat = xr.DataArray(np.linspace(-88, 88, 89), dims=["lat"])
olat.name = "lat"
olat.attrs["units"] = "degrees_north"

In [4]:
x

<xarray.DataArray 'CLDTOT_ISCCP' (time: 12, lat: 180, lon: 360)>
[777600 values with dtype=float32]
Coordinates:
  * lon      (lon) float32 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
  * lat      (lat) float32 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * time     (time) int64 1 2 3 4 5 6 7 8 9 10 11 12
Attributes:
    long_name:      total cloud fraction
    units:          1
    valid_min:      -500
    valid_max:      500
    description:    Cloud detected by either IR or VIS threshold, cloud top l...
    standard_name:  isccp_cloud_area_fraction

In [5]:
olon

<xarray.DataArray 'lon' (lon: 180)>
array([  0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,  18.,  20.,
        22.,  24.,  26.,  28.,  30.,  32.,  34.,  36.,  38.,  40.,  42.,
        44.,  46.,  48.,  50.,  52.,  54.,  56.,  58.,  60.,  62.,  64.,
        66.,  68.,  70.,  72.,  74.,  76.,  78.,  80.,  82.,  84.,  86.,
        88.,  90.,  92.,  94.,  96.,  98., 100., 102., 104., 106., 108.,
       110., 112., 114., 116., 118., 120., 122., 124., 126., 128., 130.,
       132., 134., 136., 138., 140., 142., 144., 146., 148., 150., 152.,
       154., 156., 158., 160., 162., 164., 166., 168., 170., 172., 174.,
       176., 178., 180., 182., 184., 186., 188., 190., 192., 194., 196.,
       198., 200., 202., 204., 206., 208., 210., 212., 214., 216., 218.,
       220., 222., 224., 226., 228., 230., 232., 234., 236., 238., 240.,
       242., 244., 246., 248., 250., 252., 254., 256., 258., 260., 262.,
       264., 266., 268., 270., 272., 274., 276., 278., 280., 282., 284.,
       286., 288., 290., 292., 294., 296., 298., 300., 302., 304., 306.,
       308., 310., 312., 314., 316., 318., 320., 322., 324., 326., 328.,
       330., 332., 334., 336., 338., 340., 342., 344., 346., 348., 350.,
       352., 354., 356., 358.])
Dimensions without coordinates: lon
Attributes:
    units:    degrees_east

In [6]:
olat

<xarray.DataArray 'lat' (lat: 89)>
array([-88., -86., -84., -82., -80., -78., -76., -74., -72., -70., -68.,
       -66., -64., -62., -60., -58., -56., -54., -52., -50., -48., -46.,
       -44., -42., -40., -38., -36., -34., -32., -30., -28., -26., -24.,
       -22., -20., -18., -16., -14., -12., -10.,  -8.,  -6.,  -4.,  -2.,
         0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,  18.,  20.,
        22.,  24.,  26.,  28.,  30.,  32.,  34.,  36.,  38.,  40.,  42.,
        44.,  46.,  48.,  50.,  52.,  54.,  56.,  58.,  60.,  62.,  64.,
        66.,  68.,  70.,  72.,  74.,  76.,  78.,  80.,  82.,  84.,  86.,
        88.])
Dimensions without coordinates: lat
Attributes:
    units:    degrees_north

In [7]:
# interpolate with GeoCAT's linint2
xnew = gc.linint2(x, olon, olat, xi=x.lon, yi=x.lat, icycx=1)

TypeError: 'NoneType' object is not iterable

In [ ]:
def linint_wrap(fi, xo, yo):
    """Wraps GeoCAT's linint2 function. Adds a cyclic point to ensure periodic longitude."""
    print("1")
    addpt = fi.isel(lon=0).copy()
    print("2")
    addpt['lon'] = fi.lon[-1]+(fi.lon[1]-fi.lon[0])
    print("3")
    xiwrap = xr.concat([fi, addpt], dim='lon')
    print("4")
    # if xiwrap.chunks is not None:
    #     print(f"xiwrap has chunks: {xiwrap.chunks}")
    # else:
    #     print("xiwrap is NOT chunked. --> chunk it")
    #     xiwrap = xiwrap.chunk({"lat":-1,"lon":-1})
    #     print(xiwrap.chunks)        
    return gc.linint2(xiwrap, xo, yo, xi=xiwrap.lon, yi=xiwrap.lat, icycx=1, msg_py=None)

In [ ]:

# interpolated = linint_wrap( x, olon, olat)


In [ ]:
x.values.dtype

In [ ]:
oshape = (12, len(olat), len(olat))
from geocat.f2py.linint2_wrapper import _linint2 as f2pylinint2
# _linint2(xi, yi, fi, xo, yo, icycx, msg_py, shape):
nowrap = f2pylinint2(x['lon'].values, x['lat'].values, x[0,:,:].values, olon.values, olat.values, icycx=1, msg_py=None, shape=oshape)

In [ ]:
#
# what if we did it with just numpy
#

mlon, mlat = np.meshgrid(olon, olat)

In [ ]:
# problem: nan values mess up interpolation
# try: fill in nan values and later mask those region
xfill = x.interpolate_na(dim='lon')
nmissing = np.count_nonzero(np.isnan(xfill))
print(f"N missing = {nmissing}")
if nmissing > 0:
    print("do latitude next")
    xfill = xfill.interpolate_na(dim='lat')
nmissing = np.count_nonzero(np.isnan(xfill))
print(f"N missing = {nmissing}")
if nmissing > 0:
    print("do time last")
    xfill = xfill.interpolate_na(dim='time')
nmissing = np.count_nonzero(np.isnan(xfill))
print(f"N missing = {nmissing}")
if nmissing > 0:
    xfill = xfill.ffill(dim='lon')
    xfill = xfill.bfill(dim='lon')
nmissing = np.count_nonzero(np.isnan(xfill))
print(f"N missing = {nmissing}")
if nmissing > 0:
    xfill = xfill.ffill(dim='lat')
    xfill = xfill.bfill(dim='lat')
nmissing = np.count_nonzero(np.isnan(xfill))
print(f"N missing = {nmissing}")
if nmissing > 0:
    xfill = xfill.ffill(dim='time')
    xfill = xfill.bfill(dim='time')
    print(f"N missing = {nmissing}")

# to mask final field:
msgmsk = xr.where(np.isnan(x), 0, 1)
    

# from scipy.interpolate import interp2d
from scipy.interpolate import RectSphereBivariateSpline
colatitude = np.radians(x.lat) + (np.pi / 2)  # has to be colatitude (0, pi)
ifunc = RectSphereBivariateSpline(colatitude, np.radians(x.lon), xfill[0,:,:])
# apply to new level and latitude
new_lats, new_lons = np.meshgrid(np.radians(olat.values) + (np.pi / 2), np.radians(olon.values))
xnew = ifunc.ev(new_lats, new_lons).reshape((len(olon), len(olat))).T

ifunc = RectSphereBivariateSpline(colatitude, np.radians(x.lon), msgmsk[0,:,:])
mask = ifunc.ev(new_lats, new_lons).reshape((len(olon), len(olat))).T

# convert to DataArray

xnew = xr.DataArray(xnew, dims=("lat","lon"), coords={"lat":olat, "lon":olon})

mask =  xr.DataArray(mask, dims=("lat","lon"), coords={"lat":olat, "lon":olon})

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
xnew_ma = xr.where(mask < 0.99, np.nan, xnew)

In [ ]:
xnew_ma.plot.pcolormesh()

# here 
is one

## two 

another one


### three
check
### another three
checker